# 進階的 SQL 五十道練習

> 資料查詢進階技巧

[數聚點](https://www.datainpoint.com) | 郭耀仁 <yaojenkuo@datainpoint.com>

## 資料查詢進階技巧

- 日期時間函數。
- 集合運算。
- 通用資料表運算式（CTE, Common Table Expression)。
- 交叉連接（Cross-join）與自連接（Self-join）。
- 視窗函數（Window functions）。

## （複習）SQL 的組成與分類

- 具體來說，SQL 是由保留字（Keyword）、符號、常數與函數所組合而成的一種語言。
- 依照使用目的進而區別 SQL 的分類：

|SQL 的分類|範例|
|:---------|:----|
|資料查詢語言（Data Query Language, DQL）|`SELECT ...`|
|資料定義語言（Data Definition Language, DDL）|`CREATE ...`|
|資料操作語言（Data Manipulation Language, DML）|`UPDATE ...`|
|資料控制語言（Data Control Language, DCL）|`GRANT ...`|
|交易控制語言（Transaction Control Language, TCL）|`COMMIT ...`|

## 下載範例資料

- [lookup_table.csv](https://classroom-hahow-adv-sqlfifty.s3.ap-northeast-1.amazonaws.com/lookup_table.csv)
- [time_series.csv](https://classroom-hahow-adv-sqlfifty.s3.ap-northeast-1.amazonaws.com/time_series.csv)

來源：<https://github.com/CSSEGISandData/COVID-19>

## 建立資料庫 `covid19`

```sql
CREATE DATABASE covid19 CHARACTER SET utf8mb4 COLLATE utf8mb4_unicode_ci;
```

## 匯入資料表

- 在 `covid19` 資料庫按右鍵點選 Import Data
- 調整 `pop` 資料類型為 `BIGINT`

## 日期時間函數

## 常用的 MySQL 日期時間函數

- `CURRENT_DATE()`
- `CURRENT_TIME()`
- `CURRENT_TIMESTAMP()`

```sql
SELECT CURRENT_DATE() AS current_date,
	   CURRENT_TIME() AS current_time,
	   CURRENT_TIMESTAMP() AS current_timestamp;
```

## 常用的 MySQL 日期時間函數（續）

- `ADDDATE()`
- `SUBDATE()`
- `ADDTIME()`
- `SUBTIME()`

```sql
SELECT ADDDATE(CURRENT_DATE(), 3) AS three_days_later,
       SUBDATE(CURRENT_DATE(), 2) AS two_days_ago,
	   ADDTIME(CURRENT_TIMESTAMP(), '3:00:00') AS three_hours_later,
       SUBTIME(CURRENT_TIMESTAMP(), '0:30:00') AS half_hour_ago;
```

## 常用的 MySQL 日期時間函數（續）

- `DATE_FORMAT()`
- `DATEDIFF()`

來源：<https://dev.mysql.com/doc/refman/8.0/en/date-and-time-functions.html#function_date-format>

```sql
SELECT CURRENT_DATE() AS current_date_in_iso8601,
       DATE_FORMAT(CURRENT_DATE(), '%a %b %d') AS current_date_in_custom_format,
       DATE_FORMAT(CURRENT_TIMESTAMP(), '%a %b %d %H:%i') AS current_date_in_custom_format,
       DATEDIFF(CURRENT_DATE(), '2023-01-01') AS ytd_days;
```

## 更多 MySQL 日期時間函數

來源：<https://dev.mysql.com/doc/refman/8.0/en/date-and-time-functions.html>

## 集合運算

## 什麼是集合運算

垂直關聯多個 SQL 敘述時，在具有相同的欄數和資料類型的前提下，集合運算的保留字能夠將一個查詢的結果垂直合併至另一個查詢的結果，MySQL 支援的集合運算保留字有：

- `UNION`/`UNION ALL`
- `INTERSECT`
- `EXCEPT`

## （複習）使用 `UNION` 或 `UNION ALL` 垂直關聯兩段 SQL 敘述

```sql
a SELECT Statement ...
UNION | UNION ALL
another SELECT Statement ...;
```

## `UNION` 聯集並且剔除重複值、排序

```sql
USE imdb;
SELECT name
  FROM directors
 WHERE name IN ('Steven Spielberg', 'Christopher Nolan', 'Aamir Khan')
 UNION
SELECT name
  FROM actors
 WHERE name IN ('Tom Hanks', 'Tom Cruise', 'Aamir Khan');
```

## `UNION ALL` 聯集並且保留重複值、不予排序

```sql
USE imdb;
SELECT name
  FROM directors
 WHERE name IN ('Steven Spielberg', 'Christopher Nolan', 'Aamir Khan')
 UNION ALL
SELECT name
  FROM actors
 WHERE name IN ('Tom Hanks', 'Tom Cruise', 'Aamir Khan');
```

## 使用 `INTERSECT` 垂直關聯兩段 SQL 敘述並且保留「交集」的值

```sql
a SELECT Statement ...
INTERSECT
another SELECT Statement ...;
```

```sql
SELECT name
  FROM directors
 WHERE name IN ('Steven Spielberg', 'Christopher Nolan', 'Aamir Khan')
INTERSECT
SELECT name
  FROM actors
 WHERE name IN ('Tom Hanks', 'Tom Cruise', 'Aamir Khan');
```

## 既是演員亦是導演的名單

```sql
SELECT name
  FROM actors
INTERSECT
SELECT name
  FROM directors;
```

## 使用 `EXCEPT` 垂直關聯兩段 SQL 敘述並進行「差集」運算

```sql
a SELECT Statement ...
EXCEPT
another SELECT Statement ...;
```

## `directors` 差集 `actors`

```sql
SELECT name
  FROM directors
 WHERE name IN ('Steven Spielberg', 'Christopher Nolan', 'Aamir Khan')
EXCEPT
SELECT name
  FROM actors
 WHERE name IN ('Tom Hanks', 'Tom Cruise', 'Aamir Khan');
```

## `actors` 差集 `directors`

```sql
SELECT name
  FROM actors
 WHERE name IN ('Tom Hanks', 'Tom Cruise', 'Aamir Khan')
EXCEPT
SELECT name
  FROM directors
 WHERE name IN ('Steven Spielberg', 'Christopher Nolan', 'Aamir Khan');
```

## 通用資料表運算式

## 什麼是通用資料表運算式

- 通用資料表運算式（Common Table Expression, CTE）是一種介於子查詢與檢視表之間的存在。
- 我們可以將通用資料表運算式視為一種暫存的檢視表，僅在一個 SQL 敘述中有效。
- 使用 CTE 時必須在同一個 SQL 敘述中涵蓋建立與查詢的語法。

## 建立通用資料表運算式

使用 `WITH` 建立通用資料表運算式並給予命名，然後再加入 SQL 敘述。

```sql
WITH cte_name AS (
    SQL Statement
)
SELECT columns FROM cte_name ...;
```

## 透過通用資料表運算式複習不同的關聯類型

- `(INNER) JOIN` 保留兩個資料表的「交集」觀測值。
- `LEFT JOIN` 保留左資料表的所有觀測值。
- `RIGHT JOIN` 保留右資料表的所有觀測值。
- `FULL JOIN` 保留兩個資料表的「聯集」觀測值。

## `(INNER) JOIN` 保留兩個資料表的「交集」觀測值

```sql
USE imdb;
WITH movies_shashank_forrest AS (
     SELECT *
       FROM movies
      WHERE title IN ('The Shawshank Redemption', 'Forrest Gump')
),
casting_shawshank_darkknight AS (
     SELECT *
       FROM casting
      WHERE movie_id IN (1, 3)
)
SELECT movies_shashank_forrest.title,
       casting_shawshank_darkknight.actor_id
  FROM movies_shashank_forrest
  JOIN casting_shawshank_darkknight
    ON movies_shashank_forrest.id = casting_shawshank_darkknight.movie_id;
```

## `LEFT JOIN` 保留左資料表的所有觀測值

```sql
USE imdb;
WITH movies_shashank_forrest AS (
     SELECT *
       FROM movies
      WHERE title IN ('The Shawshank Redemption', 'Forrest Gump')
),
casting_shawshank_darkknight AS (
     SELECT *
       FROM casting
      WHERE movie_id IN (1, 3)
)
SELECT movies_shashank_forrest.title,
       casting_shawshank_darkknight.actor_id
  FROM movies_shashank_forrest
  LEFT JOIN casting_shawshank_darkknight
    ON movies_shashank_forrest.id = casting_shawshank_darkknight.movie_id;
```

## `RIGHT JOIN` 保留右資料表的所有觀測值

```sql
USE imdb;
WITH movies_shashank_forrest AS (
     SELECT *
       FROM movies
      WHERE title IN ('The Shawshank Redemption', 'Forrest Gump')
),
casting_shawshank_darkknight AS (
     SELECT *
       FROM casting
      WHERE movie_id IN (1, 3)
)
SELECT movies_shashank_forrest.title,
       casting_shawshank_darkknight.actor_id
  FROM movies_shashank_forrest
 RIGHT JOIN casting_shawshank_darkknight
    ON movies_shashank_forrest.id = casting_shawshank_darkknight.movie_id;
```

## `FULL JOIN` 保留兩個資料表的「聯集」觀測值

```sql
USE imdb;
WITH movies_shashank_forrest AS (
     SELECT *
       FROM movies
      WHERE title IN ('The Shawshank Redemption', 'Forrest Gump')
),
casting_shawshank_darkknight AS (
     SELECT *
       FROM casting
      WHERE movie_id IN (1, 3)
)
SELECT movies_shashank_forrest.title,
       casting_shawshank_darkknight.actor_id
  FROM movies_shashank_forrest
 RIGHT JOIN casting_shawshank_darkknight
    ON movies_shashank_forrest.id = casting_shawshank_darkknight.movie_id
 UNION
SELECT movies_shashank_forrest.title,
       casting_shawshank_darkknight.actor_id
  FROM movies_shashank_forrest
  LEFT JOIN casting_shawshank_darkknight
    ON movies_shashank_forrest.id = casting_shawshank_darkknight.movie_id;
```

## 交叉連接與自連接

## 什麼是交叉連接

- 交叉連接（Cross-join），又稱笛卡爾連接（Cartesian join）或乘積（Product）。
- 交叉連接會將關聯的資料表中所有可能的排列組合作為結果回傳。
- 不需要使用 `ON` 描述連接鍵（Join key）。

## 使用 `CROSS JOIN` 進行交叉連接

```sql
SELECT columns
  FROM left_table
 CROSS JOIN right_table;
```

```sql
SELECT directors.name AS director_name,
       actors.name AS actor_name
  FROM directors
 CROSS JOIN actors
 WHERE directors.name IN ('Steven Spielberg', 'Christopher Nolan') AND
       actors.name IN ('Tom Hanks', 'Tom Cruise', 'Leonardo DiCaprio');
```

## 什麼是自連接

當關聯的左資料表與右資料表為同一個資料表的時候，就稱為自連接（Self-join）。

```sql
SELECT columns
  FROM table
  JOIN table
    ON table.join_key = table.join_key;
```

## 查詢相同導演的電影組合

```sql
SELECT m1.title,
	   m2.title
  FROM movies AS m1
  JOIN movies AS m2
    ON m1.director = m2.director
 WHERE m1.title != m2.title
 ORDER BY m1.title;
```

## 查詢相同上映年份的電影組合

```sql
SELECT m1.title,
	   m2.title
  FROM movies AS m1
  JOIN movies AS m2
    ON m1.release_year = m2.release_year
 WHERE m1.title != m2.title
 ORDER BY m1.title;
```

## 查詢相同評等的電影組合

```sql
SELECT m1.title,
	   m2.title
  FROM movies AS m1
  JOIN movies AS m2
    ON m1.rating = m2.rating
 WHERE m1.title != m2.title
 ORDER BY m1.rating DESC,
 	      m1.title;
```

## 視窗函數

## 什麼是視窗函數

- 視窗函數（Window functions）與分組聚合（`GROUP BY` 與聚合函數）的概念類似。
    - 兩者相同之處在於她們都是跨列的聚合計算（Aggregate across rows）。
    - 兩者相異之處在於視窗函數並不會將多列的輸出摘要為單列輸出，而聚合函數會。
- 視窗函數會搭配特別的保留字使用：
    - `OVER()`
    - `PARTITION BY`

## 如何使用視窗函數

- `PARTITION BY` 保留字作用與 `GROUP BY` 保留字類似。
- `ORDER BY` 保留字為「選填」，搭配特定視窗函數（例如具有排序功能）的時候使用。

```sql
SELECT window_function() OVER (
    PARTITION BY columns,
    ORDER BY columns,
    ...
) AS alias
  FROM table;
```

## 分組聚合與視窗函數的異同

```sql
-- 分組聚合
SELECT rating,
       COUNT(*) AS number_of_movies
  FROM movies
 GROUP BY rating;
-- 視窗函數
SELECT title,
	   rating,
       COUNT(*) OVER (
           PARTITION BY rating
       ) AS number_of_movies_by_rating
  FROM movies
 ORDER BY rating DESC;
```

## 視窗函數的分類

1. 一般聚合函數。
2. 具有排序功能的視窗函數。
3. `LEAD()` 與 `LAG()` 函數。

## 視窗函數的分類：一般聚合函數

```sql
SELECT title,
       rating,
       AVG(rating) OVER (
           PARTITION BY release_year
       ) AS avg_rating_by_year
  FROM movies
 ORDER BY release_year DESC;
```

## 視窗函數的分類：具有排序功能的視窗函數

- `ROW_NUMBER()`：相同數值給予不重複排序。
- `RANK()`：相同數值給予重複排序。
- `FIRST_VALUE()`/`LAST_VALUE()`/`NTH_VALUE()`

## 視窗函數的分類：具有排序功能的視窗函數（續）

```sql
SELECT title,
       rating,
       ROW_NUMBER() OVER (ORDER BY rating DESC) AS row_num,
       RANK() OVER (ORDER BY rating DESC) AS rating_rank
  FROM movies;
```

## 視窗函數的分類：具有排序功能的視窗函數（續）

```sql
SELECT title,
	   release_year,
       rating,
       FIRST_VALUE(rating) OVER (PARTITION BY release_year) AS highest_rating_by_year,
       LAST_VALUE(rating) OVER (PARTITION BY release_year) AS lowest_rating_by_year,
       NTH_VALUE(rating, 2) OVER (PARTITION BY release_year) AS second_highest_rating_by_year
  FROM movies
 ORDER BY release_year DESC;
```

## 視窗函數的分類：`LEAD()` 與 `LAG()` 函數



```sql
use covid19;
SELECT date,
	   country_region_id,
	   cumulative_confirmed,
	   LAG(cumulative_confirmed) OVER (
	   	   PARTITION BY country_region_id
	       ORDER BY date
	   ) AS lag_cumulative_confirmed,
	   cumulative_deaths,
	   LAG(cumulative_deaths) OVER (
	   	   PARTITION BY country_region_id
	       ORDER BY date
	   ) AS lag_cumulative_deaths
  FROM time_series
 WHERE country_region_id = (SELECT id
                              FROM lookup_table 
                             WHERE country_region = 'Taiwan');
```

## 使用 `LAG()` 函數計算每日確診、死亡數

```sql
use covid19;
SELECT date,
	   country_region_id,
	   cumulative_confirmed,
       cumulative_deaths,
	   cumulative_confirmed - LAG(cumulative_confirmed) OVER (
	   	   PARTITION BY country_region_id
	       ORDER BY date
	   ) AS daily_confirmed,
	   cumulative_deaths - LAG(cumulative_deaths) OVER (
	   	   PARTITION BY country_region_id
	       ORDER BY date
	   ) AS daily_deaths 
  FROM time_series;
```